# Dataset editing

In [1]:
import skimage.transform
import numpy as np
from sklearn.model_selection import train_test_split

In [2]:
hira = np.load("datasets/hiragana/hiragana.npz")['arr_0'].reshape([-1, 127, 128]).astype(np.float32)

In [3]:
np.max(hira)

12.0

In [4]:
hira = hira/np.max(hira)

In [5]:
# 71 characters, 160 writers, transform image to 48*48
train_images = np.zeros([71 * 160, 48, 48], dtype=np.float32)

for i in range(71 * 160):
    train_images[i] = skimage.transform.resize(hira[i], (48, 48))

arr = np.arange(71)
train_labels = np.repeat(arr, 160) # create labels

# split to train and test
train_images, test_images, train_labels, test_labels = train_test_split(train_images, train_labels, test_size=0.2)

In [6]:
np.savez_compressed("hiragana_train_images.npz", train_images)
np.savez_compressed("hiragana_train_labels.npz", train_labels)
np.savez_compressed("hiragana_test_images.npz", test_images)
np.savez_compressed("hiragana_test_labels.npz", test_labels)

# Actual training

In [2]:
import tensorflow as tf
import keras
import numpy as np
# from keras.preprocessing.image import ImageDataGenerator
from keras import backend as K

2024-03-26 18:35:34.129131: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-03-26 18:35:34.158546: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-03-26 18:35:34.691022: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [3]:
train_images = np.load("datasets/hiragana/hiragana_train_images.npz")['arr_0']
train_labels = np.load("datasets/hiragana/hiragana_train_labels.npz")['arr_0']
test_images = np.load("datasets/hiragana/hiragana_test_images.npz")['arr_0']
test_labels = np.load("datasets/hiragana/hiragana_test_labels.npz")['arr_0']

In [4]:
if K.image_data_format() == "channels_first":
  train_images = train_images.reshape(train_images.shape[0], 1,48,48)
  test_images2 = test_images.reshape(test_images.shape[0], 1,48,48)
  shape = (1,48,48)
else:
  train_images = train_images.reshape(train_images.shape[0], 48, 48, 1)
  test_images2 = test_images.reshape(test_images.shape[0], 48, 48, 1)
  shape = (48,48,1)

In [5]:
# ImageDataGenerator is deprecated; tf.keras has drop in replacements
datagen = tf.keras.preprocessing.image.ImageDataGenerator(rotation_range=15,zoom_range=0.2)
datagen.fit(train_images)

In [6]:
model = keras.Sequential([
  keras.Input(shape=shape),
  keras.layers.Conv2D(64, (3,3), activation='relu'),
  keras.layers.MaxPooling2D(2,2),
  keras.layers.Conv2D(64, (3,3), activation='relu'),
  keras.layers.MaxPooling2D(2,2),
  keras.layers.Conv2D(64, (3,3), activation='relu'),
  keras.layers.MaxPooling2D(2,2),
  keras.layers.Flatten(),
  keras.layers.Dropout(0.5),
  keras.layers.Dense(1024, activation='relu'),
  keras.layers.Dense(71, activation="softmax")
])

model.summary()

2024-03-26 18:35:43.430033: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-03-26 18:35:43.448346: W tensorflow/core/common_runtime/gpu/gpu_device.cc:2251] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 46, 46, 64)     │           640 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 23, 23, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 21, 21, 64)     │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 10, 10, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 8, 8, 64)       │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 4, 4, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 1024)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 1024)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 1024)           │     1,049,600 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 71)             │        72,775 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,196,871 (4.57 MB)

 Trainable params: 1,196,871 (4.57 MB)

 Non-trainable params: 0 (0.00 B)

In [7]:
model.compile(optimizer='adam',
              loss="sparse_categorical_crossentropy",
              metrics=['accuracy'])

In [8]:
# model.fit_generator is deprecated; model.fit now supports generators
model.fit(
    x = datagen.flow(train_images,train_labels,shuffle=True),
    epochs=30,
    validation_data=(test_images2,test_labels),
    callbacks = [
        keras.callbacks.EarlyStopping(patience=8,verbose=1,restore_best_weights=True),
        keras.callbacks.ReduceLROnPlateau(factor=0.5,patience=3,verbose=1)
    ]
)

Epoch 1/30
  8/284 ━━━━━━━━━━━━━━━━━━━━ 4s 15ms/step - accuracy: 0.0320 - loss: 4.2607  

/run/media/ren/win/Users/Ren/Projects/School/Thesis/JPHandwritingModel/venv/lib/python3.11/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:120: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


284/284 ━━━━━━━━━━━━━━━━━━━━ 4s 11ms/step - accuracy: 0.2136 - loss: 3.2636 - val_accuracy: 0.7997 - val_loss: 0.6596 - learning_rate: 0.0010
Epoch 2/30
284/284 ━━━━━━━━━━━━━━━━━━━━ 3s 10ms/step - accuracy: 0.7279 - loss: 0.9133 - val_accuracy: 0.8776 - val_loss: 0.3996 - learning_rate: 0.0010
Epoch 3/30
284/284 ━━━━━━━━━━━━━━━━━━━━ 3s 10ms/step - accuracy: 0.7997 - loss: 0.6415 - val_accuracy: 0.9208 - val_loss: 0.2447 - learning_rate: 0.0010
Epoch 4/30
284/284 ━━━━━━━━━━━━━━━━━━━━ 3s 10ms/step - accuracy: 0.8566 - loss: 0.4585 - val_accuracy: 0.9432 - val_loss: 0.1718 - learning_rate: 0.0010
Epoch 5/30
284/284 ━━━━━━━━━━━━━━━━━━━━ 3s 10ms/step - accuracy: 0.8921 - loss: 0.3263 - val_accuracy: 0.9560 - val_loss: 0.1282 - learning_rate: 0.0010
Epoch 6/30
284/284 ━━━━━━━━━━━━━━━━━━━━ 3s 10ms/step - accuracy: 0.9021 - loss: 0.3091 - val_accuracy: 0.9718 - val_loss: 0.0993 - learning_rate: 0.0010
Epoch 7/30
284/284 ━━━━━━━━━━━━━━━━━━━━ 3s 10ms/step - accuracy: 0.9203 - loss: 0.2422 - val_

In [9]:
test_loss, test_acc = model.evaluate(test_images2, test_labels)
print("Test Accuracy: ", test_acc)

71/71 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.9940 - loss: 0.0226
Test Accuracy:  0.9916373491287231


In [10]:
model.save("../models/hiragana_latest.keras")

In [41]:
keras.__version__

'3.1.1'